In [1]:
import subprocess
import time
import re

#### Function for command compatibility with windows cmd and some linux commands

In [2]:
def run_cmd(cmd, **kwargs):
    response = subprocess.Popen(
                    cmd, 
                    stderr=subprocess.PIPE, 
                    stdout=subprocess.PIPE, 
                    **kwargs)

    stdout, stderr = response.communicate()
    stdout = stdout.decode()
    stderr = stderr.decode()

    returncode = response.returncode

    # print('STDOUT BEGIN \n\n', stdout, '\nSTDOUT END\n\n\n')

    # If had parameter shell=True ignore bug of stderr on windows
    if kwargs.get('shell', False):
        return stdout
        
    else:
        if returncode != 0:
            print(f'Error code:   {returncode}')
            print(stderr)
        else:
            return stdout

## Verifying that devices are available

In [3]:
print(run_cmd('adb devices -l'))

cmd = 'adb devices -l | find "device " /c'
devices_connected = int(run_cmd(cmd, shell=True))
devices_connected

List of devices attached
192.168.1.67:5555      device product:beryllium model:POCOPHONE_F1 device:beryllium transport_id:2




1

In [4]:
def _devices_connected():
    cmd = 'adb devices -l | find "device " /c'
    if int(run_cmd(cmd, shell=True)) > 0:
        return True
    else:
        return False

## Obtaining the device data from the device list of the adb command

In [5]:
def get_devices_data():
    if not _devices_connected():
        return None

    cmd = 'adb devices -l'
    devices_list = run_cmd(cmd).strip().split('\n')[1:]

    for index, device in enumerate(devices_list):

        device = device.strip().split(' ')
        device.reverse()

        if device[4] != 'device':
            continue
        
        device_data = { data.split(':')[0] : data.split(':')[1] for data in device[0:4] }

        device_data['available'] = True if device[4] == 'device' else False
        device_data['serial'] = device[-1]

        devices_list[index] = device_data
    
    return devices_list

In [6]:
devices_list = get_devices_data()
devices_list

[{'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555'}]

### Connecting to the device console to get its IP address

In [7]:
serial = devices_list[0]['serial']
serial

'192.168.1.67:5555'

In [8]:
cmd = f'adb -s {serial} shell ip address'
print(run_cmd(cmd))

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: bond0: <BROADCAST,MULTICAST,MASTER> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether fe:85:dc:6c:b6:3e brd ff:ff:ff:ff:ff:ff
3: dummy0: <BROADCAST,NOARP,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN group default qlen 1000
    link/ether 3e:61:78:24:f8:4a brd ff:ff:ff:ff:ff:ff
    inet6 fe80::3c61:78ff:fe24:f84a/64 scope link 
       valid_lft forever preferred_lft forever
4: ip_vti0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
5: ip6_vti0@NONE: <NOARP> mtu 1500 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
6: sit0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1


In [9]:
cmd = f'adb -s {serial} shell "ip address | grep -e \'inet .* wlan\'" '
print(run_cmd(cmd))

inet 192.168.1.67/24 brd 192.168.1.255 scope global wlan0



In [10]:
cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '
cmd

'adb -s 192.168.1.67:5555 shell "ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

In [11]:
ip = run_cmd(cmd)
print(ip)

ip = ip.split('/')[0]
print(ip)

192.168.1.67/24

192.168.1.67


In [12]:
def get_device_ip(serial = None):
    if serial is None:
        serial = get_devices_data()[0]['serial']

    cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    return run_cmd(cmd).split('/')[0]

In [13]:
print(get_device_ip())
print(get_device_ip(serial=serial))

192.168.1.67
192.168.1.67


In [14]:
devices_list[0]['ip'] = get_device_ip()
devices_list

[{'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555',
  'ip': '192.168.1.67'}]

In [15]:
def tcpip_connect(port = 5555, serial = None, ip = None):
    port = str(port)
    ip = get_device_ip(serial) if ip is None else ip
    
    cmd = f'adb tcpip {port}' if serial is None else f'adb -s {serial} tcpip {port}'
    print(run_cmd(cmd))

    cmd = f'adb connect {ip}:{port}'
    print(run_cmd(cmd))


In [16]:
tcpip_connect()
time.sleep(2)


restarting in TCP mode port: 5555

already connected to 192.168.1.67:5555



In [17]:
devices_list = get_devices_data()
devices_list

[{'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555'}]

In [18]:
for index, device in enumerate(devices_list):
    devices_list[index]['ip'] = get_device_ip(device['serial'])

devices_list

[{'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555',
  'ip': '192.168.1.67'}]

In [19]:
get_devices_data()[0]['serial']

'192.168.1.67:5555'

In [20]:
def run_adb(cmd, serial=None):
    if not serial:
        devices = get_devices_data()
        if not devices:
            return None
        else:
            serial = devices[0]['serial']

    sh = f'adb -s {serial} shell '
    return run_cmd(sh + cmd)


In [21]:
serial = devices_list[0]['serial']


In [22]:
cmd = 'find --help'

print(run_adb(cmd, serial))

usage: find [-HL] [DIR...] [<options>]

Search directories for matching files.
Default: search "." match all -print all matches.

-H  Follow command line symlinks         -L  Follow all symlinks

Match filters:
-name  PATTERN  filename with wildcards   -iname      case insensitive -name
-path  PATTERN  path name with wildcards  -ipath      case insensitive -path
-user  UNAME    belongs to user UNAME     -nouser     user ID not known
-group GROUP    belongs to group GROUP    -nogroup    group ID not known
-perm  [-/]MODE permissions (-=min /=any) -prune      ignore contents of dir
-size  N[c]     512 byte blocks (c=bytes) -xdev       only this filesystem
-links N        hardlink count            -atime N[u] accessed N units ago
-ctime N[u]     created N units ago       -mtime N[u] modified N units ago
-newer FILE     newer mtime than FILE     -mindepth # at least # dirs down
-depth          ignore contents of dir    -maxdepth # at most # dirs down
-inum  N        inode number N         

In [23]:
# Searching for all the files that do not have the extension .mp3, .m4a and .flac (without distinguishing capitals)
cmd = 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'
print(run_adb(cmd))

# Deleting that files
cmd = 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac -delete'
print(run_adb(cmd))

cmd = 'find sdcard/musica/ -type f ! -iname *.mp3 -a ! -iname *.m4a -a ! -iname *.flac | wc -l'
print(run_adb(cmd))


0


0



In [24]:
# Now with empty folders
cmd = 'find sdcard/musica/ -type d -empty | wc -l'
print(run_adb(cmd))

cmd = 'find sdcard/musica/ -type d -empty'
print(run_adb(cmd))

cmd = 'find sdcard/musica/ -type d -empty -delete'
print(run_adb(cmd))

cmd = 'find sdcard/musica/ -type d -empty | wc -l'
print(run_adb(cmd))

0



0



In [25]:
path = '/sdcard/musica/AC\ DC/'
cmd = f'ls -p {path} -R'
artist_content = run_adb(cmd).strip()
print(artist_content)

/sdcard/musica/AC DC/:
74 Jailbreak (1984)/
AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]/
Back In Black (Australian Editions) (1980)/
Backtracks (2009)/
Ballbreaker (1995)/
Black Ice (2008)/
Blow Up Your Video (Australian Editions) (1988)/
Dirty Deeds Done Dirt Cheap (Australian Edition) (1976)/
Flick Of The Switch (Australian Editions) (1983)/
Fly On The Wall (Australian Editions) (1985)/
For Those About To Rock (We Salute You) (1981)/
High Voltage (Australian Editions) (1975)/
Highway To Hell (Australian Editions) (1979)/
If You Want Blood You've Got It (Australian Editions) (1978)/
Powerage (Australian Editions) (1978)/
Stiff Upper Lip (2000)/
T.N.T (Australian Editions) (1975)/
The Razor's Edge (Australian Editions) (1990)/
Who Made Who (Australian Editions) (1986)/
test/

/sdcard/musica/AC DC/74 Jailbreak (1984):
01 - Jailbreak.mp3
02 - You Aint Got A Hold On Me.mp3
03 - Show Business.mp3
04 - Soul Stripper.mp3
05 - Baby Please Dont Go.mp3

/sdcard/musica/AC DC/AC_DC - Rock or Bus

In [26]:
artist_content = artist_content.replace('\n', '')
artist_content = artist_content.split('\r')

# deleting empty items
artist_content = [item for item in artist_content if item]
artist_content


['/sdcard/musica/AC DC/:',
 '74 Jailbreak (1984)/',
 'AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]/',
 'Back In Black (Australian Editions) (1980)/',
 'Backtracks (2009)/',
 'Ballbreaker (1995)/',
 'Black Ice (2008)/',
 'Blow Up Your Video (Australian Editions) (1988)/',
 'Dirty Deeds Done Dirt Cheap (Australian Edition) (1976)/',
 'Flick Of The Switch (Australian Editions) (1983)/',
 'Fly On The Wall (Australian Editions) (1985)/',
 'For Those About To Rock (We Salute You) (1981)/',
 'High Voltage (Australian Editions) (1975)/',
 'Highway To Hell (Australian Editions) (1979)/',
 "If You Want Blood You've Got It (Australian Editions) (1978)/",
 'Powerage (Australian Editions) (1978)/',
 'Stiff Upper Lip (2000)/',
 'T.N.T (Australian Editions) (1975)/',
 "The Razor's Edge (Australian Editions) (1990)/",
 'Who Made Who (Australian Editions) (1986)/',
 'test/',
 '/sdcard/musica/AC DC/74 Jailbreak (1984):',
 '01 - Jailbreak.mp3',
 '02 - You Aint Got A Hold On Me.mp3',
 '03 - Show Business.

In [27]:
folders = []
for index, item in enumerate(artist_content):
    if item[-1] == ':':
        folders.append( (item.split('/')[-1][:-1], index) )
        
folders

[('', 0),
 ('74 Jailbreak (1984)', 21),
 ('AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]', 27),
 ('Back In Black (Australian Editions) (1980)', 34),
 ('Backtracks (2009)', 44),
 ('CD1 Studio Rarities', 46),
 ('Ballbreaker (1995)', 55),
 ('Black Ice (2008)', 63),
 ('Blow Up Your Video (Australian Editions) (1988)', 72),
 ('Dirty Deeds Done Dirt Cheap (Australian Edition) (1976)', 78),
 ('Flick Of The Switch (Australian Editions) (1983)', 82),
 ('Fly On The Wall (Australian Editions) (1985)', 87),
 ('For Those About To Rock (We Salute You) (1981)', 96),
 ('High Voltage (Australian Editions) (1975)', 103),
 ('Highway To Hell (Australian Editions) (1979)', 108),
 ("If You Want Blood You've Got It (Australian Editions) (1978)", 116),
 ('Powerage (Australian Editions) (1978)', 119),
 ('Stiff Upper Lip (2000)', 125),
 ('T.N.T (Australian Editions) (1975)', 132),
 ("The Razor's Edge (Australian Editions) (1990)", 138),
 ('Who Made Who (Australian Editions) (1986)', 148),
 ('test', 154),
 ('fold

In [28]:
print(folders[1][0])

slice_begin = folders[1][1] + 1
slice_end = folders[1+1][1]
print(f'{slice_begin} : {slice_end}')

print( artist_content[slice_begin:slice_end] )

74 Jailbreak (1984)
22 : 27
['01 - Jailbreak.mp3', '02 - You Aint Got A Hold On Me.mp3', '03 - Show Business.mp3', '04 - Soul Stripper.mp3', '05 - Baby Please Dont Go.mp3']


In [29]:
tree = {}
subfold_tmp = []
for index, folder in enumerate( folders ):
    try:
        next_folder = folders[index + 1]
        slice_end = next_folder[1]
    except IndexError:
        slice_end = len(artist_content)

    slice_begin = folder[1] + 1
    if len(subfold_tmp) == 0:
        if slice_end - slice_begin != 1:
            key = folder[0]
        else:
            subfold_tmp.append(folder[0] + '/' + next_folder[0])
            continue    
    else:
        key = subfold_tmp[0]
        subfold_tmp.pop()
     
    value = artist_content[slice_begin:slice_end]
    tree[ key ] = value

tree.pop('', None)
tree

{'74 Jailbreak (1984)': ['01 - Jailbreak.mp3',
  '02 - You Aint Got A Hold On Me.mp3',
  '03 - Show Business.mp3',
  '04 - Soul Stripper.mp3',
  '05 - Baby Please Dont Go.mp3'],
 'AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]': ['01. Rock or Bust.mp3',
  '04. Miss Adventure.mp3',
  '05. Dogs of War.mp3',
  '08. Baptism By Fire.mp3',
  '09. Rock The House.mp3',
  '10. Sweet Candy.mp3'],
 'Back In Black (Australian Editions) (1980)': ['01 Hells Bells.mp3',
  '02 Shoot To Thrill.mp3',
  '04 Giving The Dog A Bone.mp3',
  '05 Let Me Put My Love Into You.mp3',
  '06 Back In Black.mp3',
  '07 You Shook Me All Night Long.mp3',
  '08 Have A Drink On Me.mp3',
  '09 Shake A Leg.mp3',
  "10 Rock'n' Roll Ain't Noise Pollution.mp3"],
 'Backtracks (2009)/CD1 Studio Rarities': ['A01 Stick Around.mp3',
  'A03 Fling Thing.mp3',
  'A04 R.I.P. (Rock In Peace).mp3',
  'A06 Crabsody In Blue.mp3',
  'A09 Borrowed Time.mp3',
  'A10 Down On The Borderline.mp3',
  'A11 Big Gun.mp3',
  'A12 Cyberspace.mp3'],
 'Ba

In [30]:
txt = tree['Stiff Upper Lip (2000)'][0]
re.sub(r'^\d+\s*[.-]?\s*', '', txt)

'Stiff Upper Lip.mp3'

In [31]:
tree_modify = {}
for folder in tree:
    files = []
    for file in tree[folder]: 
        modify = re.sub(r'^\d+\s*[.-]?\s*', '', file)
        files.append(modify)

    tree_modify[folder] = files
    
tree_modify
        
    

{'74 Jailbreak (1984)': ['Jailbreak.mp3',
  'You Aint Got A Hold On Me.mp3',
  'Show Business.mp3',
  'Soul Stripper.mp3',
  'Baby Please Dont Go.mp3'],
 'AC_DC - Rock or Bust (2014) [MP3 @ 320 kbps]': ['Rock or Bust.mp3',
  'Miss Adventure.mp3',
  'Dogs of War.mp3',
  'Baptism By Fire.mp3',
  'Rock The House.mp3',
  'Sweet Candy.mp3'],
 'Back In Black (Australian Editions) (1980)': ['Hells Bells.mp3',
  'Shoot To Thrill.mp3',
  'Giving The Dog A Bone.mp3',
  'Let Me Put My Love Into You.mp3',
  'Back In Black.mp3',
  'You Shook Me All Night Long.mp3',
  'Have A Drink On Me.mp3',
  'Shake A Leg.mp3',
  "Rock'n' Roll Ain't Noise Pollution.mp3"],
 'Backtracks (2009)/CD1 Studio Rarities': ['A01 Stick Around.mp3',
  'A03 Fling Thing.mp3',
  'A04 R.I.P. (Rock In Peace).mp3',
  'A06 Crabsody In Blue.mp3',
  'A09 Borrowed Time.mp3',
  'A10 Down On The Borderline.mp3',
  'A11 Big Gun.mp3',
  'A12 Cyberspace.mp3'],
 'Ballbreaker (1995)': ['Hard As A Rock.mp3',
  'The Furor.mp3',
  'Boogie Man.m